<a href="https://colab.research.google.com/github/gurucl/apache_beam_tutorial/blob/main/Apache_Beam_Workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world!')

In [ ]:
! pip3 install apache_beam

In [ ]:
! pwd
! ls -lrt sample_data

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()
output = ( p1 
          | beam.io.ReadFromText("/content/sample_data/tiny_movies.csv", skip_header_lines=1)
          | beam.Map(lambda x: x.split(","))
       #   | beam.Sample.FixedSizeGlobally(5)
         | beam.Filter(lambda x: float(x[3].strip()) > 8.8)
        # | beam.io.WriteToText("sample_data/output", "result", num_shards=3, header = "Movie ID, Movie_Name, Year, Rating", compression_type = beam.io.filesystem.CompressionTypes.GZIP )
         | beam.Map(print)
          )
p1.run()

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()

schema1 = """ {
   "type" : "record",
   "namespace" : "Tutorialspoint",
   "name" : "Employee",
   "fields" : [
      { "name" : "Movie_id" , "type" : "string" },
      { "name" : "Name" , "type" : "string" },
      { "name" : "Year" , "type" : "string" },
      { "name" : "Rating" , "type" : "string" }
   ]
}"""
output = ( p1 
          | beam.io.ReadFromText("/content/sample_data/tiny_movies.csv", skip_header_lines=1)
          | beam.Map(lambda x: x.split(","))
       #   | beam.Sample.FixedSizeGlobally(5)
         | beam.Filter(lambda x: float(x[3].strip()) > 8.8)
        # | beam.io.WriteToText("sample_data/output", "result", shard_name_template='-SS-NN', num_shards=3, header = "Movie ID, Movie_Name, Year, Rating", )
          | beam.io.WriteToAvro("out", schema = schema1)
        # | beam.Map(print)
          )
p1.run()

In [ ]:
! cat sample_data/tiny_movies.csv

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

result = (p1
        | beam.Create([1,2,3,4,5,6,7,8,9,10])
      #   | beam.Filter(lambda x: x%2==0)
       | beam.Map(print)
          )
p1.run()



In [ ]:
import apache_beam as beam
import re

p1 = beam.Pipeline()

list = ["K", "G", "F", "Kolar", "and", "to"]

def get_details(str):
  return True if str in list else False
  # if str in list:
  #   return True

class FlattenData(beam.DoFn):
  def process(self, element):
    return re.findall('[a-zA-Z]+', element)

class FilterData(beam.DoFn):
  def process(self, element):
    if element in list:
      return [element]
  
class GroupeData(beam.DoFn):
  def process(self, element):
    return [(element,1)]
  

class CalcResult(beam.DoFn):
  def process(self, element):
    (key, value) = element
    return [(key, sum(value))]

result = (
    p1
    | beam.io.ReadFromText("/content/sample_data/kgf_2.txt")
  # | beam.FlatMap(lambda x: re.findall('[a-zA-Z]+', x))
    | beam.ParDo(FlattenData())
  # | beam.Filter(get_details)
    | beam.ParDo(FilterData())
  # | beam.Map(lambda x: (x,1))
    | beam.ParDo(GroupeData())
  # | beam.CombinePerKey(sum)
    | beam.GroupByKey()
    | beam.ParDo(CalcResult())
     | beam.Map(print)
  #  | beam.io.WriteToText("/content/sample_data/words_output")
    
)
p1.run()



In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

movies_data = [
               (1,"Avenger"),
               (2,"Infinity War"),
               (3,"Winter Soldier"),
               (4,"Thor - dark World")
]
ratings_data = [
                (1, 8.5),
                (2, 9.1),
                (3, 7.5),
                (1, 8.9),
                (2, 9.2) 
]

movies = p1 |'reading Movies data' >> beam.Create(movies_data)
ratings = p1 |'reading Ratings data' >> beam.Create(ratings_data)

# joined_data = ( {'movie_name':movies, 'movie_ratings':ratings} 
#                |  beam.CoGroupByKey())  | beam.Map(print)
#              #  | beam.Map(lambda x: (x[0], x[1][1][0], x[1][0]))
              
joined_data = ( {movies, ratings} 
               |  beam.CoGroupByKey()) | beam.Map(lambda x: (x[0], x[1][1][0], x[1][0], sum(x[1][0]), sum(x[1][0])/len(x[1][0]) if len(x[1][0])!=0 else 0 ))| beam.Map(print)

p1.run()

In [ ]:
import apache_beam as beam


p1 = beam.Pipeline()

def my_partition_fn(element, num_partitions):
  return 0 if element%2==0 else 1

result = (p1
          | beam.Create(range(1,11))
          | beam.Partition(my_partition_fn, 2)
        )
# result[1] | 'Printing records from First Partition' >> beam.Map(print)
result[0] | 'Printing records from Second Partition' >>  beam.Map(print)

p1.run()

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

odd = [1,3,5,7,9]
even = [2,4,6,8]
name = ['Ram', 'Rahim', 'Robert' ]

odd_p1 = (p1| 'creating Odd PCollection' >> beam.Create(odd))
even_p1 = (p1| 'creating Even PCollection' >> beam.Create(even))
name_p1 = (p1| 'creating Name PCollection' >> beam.Create(name))

( {odd_p1, even_p1, name_p1} | beam.Flatten()) | beam.Map(print)

p1.run()




In [32]:
import apache_beam as beam

p1 = beam.Pipeline()

class My_PTransform(beam.PTransform):
  def expand(self, input_collection):
    result = (input_collection | beam.Map(lambda x: f"{x[0]} has received {x[2]} marks"))
    return result

students_data = (p1
                | beam.io.ReadFromText("/content/sample_data/student_marks.csv")
                | beam.Map(lambda x: x.split(","))
                | beam.Map(lambda x: (x[0], x[1], int(x[2]) + int(x[3]) + int(x[4]) ))
                )

us_students_data = (students_data 
                    |'Filtering US Student records' >> beam.Filter(lambda x: x[1].strip() == 'US') 
                 #   | 'Printing the US Student Records' >> beam.Map(print)
                # | beam.Map(lambda x: f"{x[0]} has received {x[2]} marks")
                | 'Applying My transformations to US student Records' >> My_PTransform()
                 | 'Writing US Student Records' >> beam.io.WriteToText("/content/sample_data/us_students_data", num_shards = 1)
                    )

in_students_data = (students_data 
                    | 'Filtering IN Student records' >> beam.Filter(lambda x: x[1].strip() == 'IN') 
                  #  | 'Printing the IN Student Records' >>  beam.Map(print)
                 # | beam.Map(lambda x: f"{x[0]} has received {x[2]} marks")
                 | 'Applying My transformations to IN student Records' >> My_PTransform()
                  |'Writing IN Student Records' >> beam.io.WriteToText("/content/sample_data/in_students_data", num_shards = 1)
                    )

p1.run()

In [67]:
from apache_beam import pvalue
from apache_beam.typehints.decorators import with_output_types
import apache_beam as beam

p1 = beam.Pipeline()

side_list = []
with open("/content/sample_data/exclude_employee.txt", 'r') as file:
  for id in file:
    side_list.append(int(id.strip()))
print(side_list)

class SpitRows(beam.DoFn):
  def process(self, element):
    return [element.split(",")]

class GetFields(beam.DoFn):
  def process(self, element, ex_data):
    fields = element.split(",")
    if int(fields[0]) not in ex_data:
      return [fields]

class PartitionedData(beam.DoFn):
  def process(self, element, dept, sal):
    if element[4] == dept:
      yield beam.pvalue.TaggedOutput('DPL_Data', element)
    else:
      yield beam.pvalue.TaggedOutput('Non_DPL_Data', element)
    if int(element[3]) > sal:
      yield [element]


employee_data = (p1
                 | beam.io.ReadFromText("/content/sample_data/employee.csv", skip_header_lines=1)
              #  | beam.ParDo(GetFields(), side_list)
              #  | beam.io.WriteToText("sample_data/employee_output_data")
                 | beam.ParDo(SpitRows())
                 | beam.ParDo(PartitionedData(), dept='DPL', sal=1000).with_outputs('DPL_Data', 'Non_DPL_Data', main='Elites')
                 )

DPL_Data = employee_data.DPL_Data
Non_DPL_Data = employee_data.Non_DPL_Data
Elites_Data = employee_data.Elites

DPL_Data | 'Writing DPL Data' >> beam.io.WriteToText('sample_data/dpl_data', num_shards=1)
Non_DPL_Data | 'Writing Non-DPL Data' >> beam.io.WriteToText('sample_data/non_dpl_data', num_shards=1)
Elites_Data | 'Writing Elites Data' >> beam.io.WriteToText('sample_data/elites_data', num_shards=1)

p1.run()


[1, 3, 7]


In [66]:
! pwd
! ls /content/sample_data
# ! rm -rf /content/sample_data/beam-temp-*

/content
anscombe.json			     kgf_2.txt
california_housing_test.csv	     mnist_test.csv
california_housing_train.csv	     mnist_train_small.csv
dpl_data-00000-of-00001		     non_dpl_data-00000-of-00001
elites_data-00000-of-00001	     README.md
employee.csv			     student_marks.csv
employee_output_data-00000-of-00001  tiny_movies.csv
exclude_employee.txt		     us_students_data-00000-of-00001
in_students_data-00000-of-00001
